NOVA VERSÂO AGORA COM InceptionV3Model

In [1]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "./data/train"
VALIDATION_DIR = "./data/valid"

training_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.15,horizontal_flip=True,fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR, target_size=(224,224), shuffle=True, class_mode='categorical', batch_size=128)
validator_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, target_size=(224,224), shuffle=True, class_mode='categorical', batch_size=14)

Found 6166 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

input_tensor = Input(shape=(224,224,3))
base_model = InceptionV3(input_tensor = input_tensor, weights='imagenet', include_top = False)



87924736/87910968 [==============================] - 407s 5us/step
